<a href="https://colab.research.google.com/github/Hoopsforever/AlphaFold-capabilities-with-nanobody-antigen-complex-prediction/blob/main/Count_contacts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Step 1: Install the necessary libraries
!pip install --upgrade gspread gspread-dataframe


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Step 2: Read the excel file with pandas
import pandas as pd

filepath = "/content/drive/MyDrive/Rishi_project_files/Compare_Contacts/orig_vs_model_contacts.xlsx" # replace with your file path
df = pd.read_excel(filepath)

df_slice = df.iloc[0:40]

counter_real = 0
# Iterate over rows
for index, row in df_slice.iterrows():
    # Iterate over columns in this row
    for column in df.columns:
        cell_value1 = row[column]

        if type(cell_value1) != str and cell_value1 > 0:
          counter_real += 1

print(f"Avg # of real contact positions: {counter_real/40}")
print(counter_real)

df_slice2 = df.iloc[41:83]

counter_pred = 0
# Iterate over rows
for index, row in df_slice2.iterrows():
    # Iterate over columns in this row
    for column in df.columns:
        cell_value2 = row[column]
        if type(cell_value2) != str and cell_value2 > 0:
          counter_pred += 1

print(f"Avg # of predicted contact positions: {counter_pred/40}")
print(counter_pred)


In [ ]:
contacts = pd.read_excel("/content/drive/MyDrive/Rishi_project_files/Compare_Contacts/orig_vs_model_contacts.xlsx",index_col = 0)
df_real = contacts.iloc[0:40, :]
df_pred = contacts.iloc[41:81, :]


correct_contacts = 0
for i in range(0,df_real.shape[0]): # x-axis of real
  for j in range(0,df_real.shape[1]): # y-axis of real
    if isinstance(df_real.iloc[i,j], str): # is this cell in the real section a string like 7e53?
      continue
    elif df_real.iloc[i,j] > 0: # if not, then is it bigger than 0?
      if isinstance(df_pred.iloc[i,j], str):  # if it is, then for that same i,j coordinate is the cell in df_pred a string?
        continue
      elif df_pred.iloc[i,j] > 0: # if not, then is the cell in df_pred also bigger than 0?
        correct_contacts += 1 # if it is, then add 1 to correct_contacts


print(correct_contacts)
real_sensitivity = (correct_contacts/counter_real)
pred_sensitivity = (correct_contacts/counter_pred)

print(f"Real Sensitivity: {real_sensitivity}")
print(f"Pred Sensitivity: {pred_sensitivity}")




In [ ]:
sequence = pd.read_excel("/content/drive/MyDrive/Rishi_project_files/Compare_Contacts/orig_contacts.xlsx",index_col = 0)


real_seq = sequence.iloc[0:42, :]
df_real = contacts.iloc[0:40, :]
df_pred = contacts.iloc[41:81, :]


df = []
for i in range(0,df_real.shape[0]):
  new = []
  for j in range(0,df_real.shape[1]):
    if isinstance(df_real.iloc[i,j],str): # is this particular cell at i,j a string?
      new.append("x") # if so, then add an x at that cell to new
    elif df_real.iloc[i,j] == 0: # if not, then check if that same cell in the real section of contacts is a 0.
      if df_pred.iloc[i,j] == 0: # if it is, then check if the corresponding cell in the pred section is also a 0
        new.append(" ") # add a blank to new if it is
      elif df_pred.iloc[i,j] > 0: # otherwise, check if the pred cell is bigger than 0
        new.append(real_seq.iloc[i,j].lower()) # These are our False Positives
    elif df_real.iloc[i,j] > 0:
      if df_pred.iloc[i,j] == 0:
        new.append(real_seq.iloc[i,j].upper()) # False negatives
      elif type(df_pred.iloc[i,j]) != str and df_pred.iloc[i,j] > 0:
        new.append(" ")

  df.append(new)

columns = [str(i+1) for i in range(145)]

mistakes = pd.DataFrame(df,columns = columns)
mistakes.index = real_seq.index
mistakes.columns = real_seq.columns

# Save the DataFrame to an Excel file
#mistakes.to_excel("/content/drive/MyDrive/Colab Notebooks/Rishi_project_files/Compare_Contacts/mistakes.xlsx")


In [ ]:
import pandas as pd

mistake_map = pd.read_excel("/content/drive/MyDrive/Rishi_project_files/Compare_Contacts/mistakes.xlsx",index_col = 0)

df_mistake = mistake_map.iloc[0:42, :]

# Flatten the DataFrame to a Series
mistakes = df_mistake.values.flatten()
no_x = mistakes[mistakes != 'x']

# Count the occurrences of each value
residues = []
for i in range(0,df_mistake.shape[0]):
  for j in range(0,df_mistake.shape[1]):
    if type(df_mistake.iloc[i,j]) != str or df_mistake.iloc[i,j] == 'x':
      wrong_res = pd.Series(no_x).value_counts()
residues.append(wrong_res)

residue_series = residues[0] #get the contents of the residues list

df = residue_series.reset_index()

# Rename the columns
df.columns = ['Residue', 'Count']

print(df)


In [ ]:
import pandas as pd

contacts = pd.read_excel("/content/drive/MyDrive/Rishi_project_files/Compare_Contacts/orig_vs_model_contacts.xlsx",index_col = 0)
df_real = contacts.iloc[0:40, :]
df_pred = contacts.iloc[41:81, :]

sequence = pd.read_excel("/content/drive/MyDrive/Rishi_project_files/Compare_Contacts/orig_contacts.xlsx",index_col = 0)
real_seq = sequence.iloc[0:42, :]





contact_columns = []
for i in range(0,df_real.shape[0]): # x-axis of real
  for j in range(0,df_real.shape[1]): # y-axis of real
    if isinstance(df_real.iloc[i,j], str): # is this cell in the real section a string like 7e53?
      continue
    elif df_real.iloc[i,j] > 0: # if not, then is it bigger than 0?
      contact_columns.append(j)
    elif df_pred.iloc[i,j] > 0:
      contact_columns.append(j)

a = list(set(contact_columns))


z = []
for i in range(0,40):
  counter = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
  for j in a:
    if real_seq.iloc[i,j] == "A":
      counter[0] += 1
    if real_seq.iloc[i,j] == "C":
      counter[1] += 1
    if real_seq.iloc[i,j] == "D":
      counter[2] += 1
    if real_seq.iloc[i,j] == "E":
      counter[3] += 1
    if real_seq.iloc[i,j] == "F":
      counter[4] += 1
    if real_seq.iloc[i,j] == "G":
      counter[5] += 1
    if real_seq.iloc[i,j] == "H":
      counter[6] += 1
    if real_seq.iloc[i,j] == "I":
      counter[7] += 1
    if real_seq.iloc[i,j] == "K":
      counter[8] += 1
    if real_seq.iloc[i,j] == "L":
      counter[9] += 1
    if real_seq.iloc[i,j] == "M":
      counter[10] += 1
    if real_seq.iloc[i,j] == "N":
      counter[11] += 1
    if real_seq.iloc[i,j] == "P":
      counter[12] += 1
    if real_seq.iloc[i,j] == "Q":
      counter[13] += 1
    if real_seq.iloc[i,j] == "R":
      counter[14] += 1
    if real_seq.iloc[i,j] == "S":
      counter[15] += 1
    if real_seq.iloc[i,j] == "T":
      counter[16] += 1
    if real_seq.iloc[i,j] == "V":
      counter[17] += 1
    if real_seq.iloc[i,j] == "W":
      counter[18] += 1
    if real_seq.iloc[i,j] == "Y":
      counter[19] += 1
  z.append(counter)



contact_AAs = pd.DataFrame(z,columns = ("A","C","D","E","F","G","H","I","K","L","M","N","P","Q","R","S","T","V","W","Y"))
contact_AAs.index = df_real.index

contact_AAs




In [ ]:
import pandas as pd

mistake_map = pd.read_excel("/content/drive/MyDrive/Rishi_project_files/Compare_Contacts/mistakes.xlsx",index_col = 0)

df_mistake = mistake_map.iloc[0:42, :]

pos=[]
neg=[]
for i in range(0,df_mistake.shape[0]):
  counter_pos = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
  counter_neg = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
  for j in range(0,df_mistake.shape[1]):
    if df_mistake.iloc[i,j] == "A":
      counter_neg[0] += 1
    if df_mistake.iloc[i,j] == "C":
      counter_neg[1] += 1
    if df_mistake.iloc[i,j] == "D":
      counter_neg[2] += 1
    if df_mistake.iloc[i,j] == "E":
      counter_neg[3] += 1
    if df_mistake.iloc[i,j] == "F":
      counter_neg[4] += 1
    if df_mistake.iloc[i,j] == "G":
      counter_neg[5] += 1
    if df_mistake.iloc[i,j] == "H":
      counter_neg[6] += 1
    if df_mistake.iloc[i,j] == "I":
      counter_neg[7] += 1
    if df_mistake.iloc[i,j] == "K":
      counter_neg[8] += 1
    if df_mistake.iloc[i,j] == "L":
      counter_neg[9] += 1
    if df_mistake.iloc[i,j] == "M":
      counter_neg[10] += 1
    if df_mistake.iloc[i,j] == "N":
      counter_neg[11] += 1
    if df_mistake.iloc[i,j] == "P":
      counter_neg[12] += 1
    if df_mistake.iloc[i,j] == "Q":
      counter_neg[13] += 1
    if df_mistake.iloc[i,j] == "R":
      counter_neg[14] += 1
    if df_mistake.iloc[i,j] == "S":
      counter_neg[15] += 1
    if df_mistake.iloc[i,j] == "T":
      counter_neg[16] += 1
    if df_mistake.iloc[i,j] == "V":
      counter_neg[17] += 1
    if df_mistake.iloc[i,j] == "W":
      counter_neg[18] += 1
    if df_mistake.iloc[i,j] == "Y":
      counter_neg[19] += 1

# -----------------------------------------------------------------BELOW IS WHERE I CHECK FOR THE FALSE POSITIVES

    if df_mistake.iloc[i,j] == "a":
      counter_pos[0] += 1
    if df_mistake.iloc[i,j] == "c":
      counter_pos[1] += 1
    if df_mistake.iloc[i,j] == "d":
      counter_pos[2] += 1
    if df_mistake.iloc[i,j] == "e":
      counter_pos[3] += 1
    if df_mistake.iloc[i,j] == "f":
      counter_pos[4] += 1
    if df_mistake.iloc[i,j] == "g":
      counter_pos[5] += 1
    if df_mistake.iloc[i,j] == "h":
      counter_pos[6] += 1
    if df_mistake.iloc[i,j] == "i":
      counter_pos[7] += 1
    if df_mistake.iloc[i,j] == "j":
      counter_pos[8] += 1
    if df_mistake.iloc[i,j] == "k":
      counter_pos[9] += 1
    if df_mistake.iloc[i,j] == "l":
      counter_pos[10] += 1
    if df_mistake.iloc[i,j] == "n":
      counter_pos[11] += 1
    if df_mistake.iloc[i,j] == "p":
      counter_pos[12] += 1
    if df_mistake.iloc[i,j] == "q":
      counter_pos[13] += 1
    if df_mistake.iloc[i,j] == "r":
      counter_pos[14] += 1
    if df_mistake.iloc[i,j] == "s":
      counter_pos[15] += 1
    if df_mistake.iloc[i,j] == "t":
      counter_pos[16] += 1
    if df_mistake.iloc[i,j] == "v":
      counter_pos[17] += 1
    if df_mistake.iloc[i,j] == "w":
      counter_pos[18] += 1
    if df_mistake.iloc[i,j] == "y":
      counter_pos[19] += 1


  pos.append(counter_pos)
  neg.append(counter_neg)


False_neg = pd.DataFrame(neg,columns = ("A","C","D","E","F","G","H","I","K","L","M","N","P","Q","R","S","T","V","W","Y"))
False_neg.index = df_real.index

False_pos = pd.DataFrame(pos,columns = ("a","c","d","e","f","g","h","i","k","l","m","n","p","q","r","s","t","v","w","y"))
False_pos.index = df_real.index



In [ ]:
False_neg

In [ ]:
False_pos

In [ ]:
divide = False_neg.div(contact_AAs)
divide

In [ ]:

False_pos.columns = False_neg.columns

divide2 = False_pos.div(contact_AAs)
divide2

In [ ]:
import pandas as pd

contacts = pd.read_excel("/content/drive/MyDrive/Rishi_project_files/Compare_Contacts/orig_vs_model_contacts.xlsx",index_col = 0)
df_real = contacts.iloc[0:40, :]
df_pred = contacts.iloc[41:81, :]

sequence = pd.read_excel("/content/drive/MyDrive/Rishi_project_files/Compare_Contacts/orig_contacts.xlsx",index_col = 0)
real_seq = sequence.iloc[0:42, :]


real = []
for i in range(0,df_real.shape[0]):
  counter_real = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
  counter_pred = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
  for j in range(0,df_real.shape[1]):
    if isinstance(df_real.iloc[i,j],str):
      continue
    elif df_real.iloc[i,j] > 0:
      if real_seq.iloc[i,j] == "A":
        counter_real[0] += 1
      if real_seq.iloc[i,j] == "C":
        counter_real[1] += 1
      if real_seq.iloc[i,j] == "D":
        counter_real[2] += 1
      if real_seq.iloc[i,j] == "E":
        counter_real[3] += 1
      if real_seq.iloc[i,j] == "F":
        counter_real[4] += 1
      if real_seq.iloc[i,j] == "G":
        counter_real[5] += 1
      if real_seq.iloc[i,j] == "H":
        counter_real[6] += 1
      if real_seq.iloc[i,j] == "I":
        counter_real[7] += 1
      if real_seq.iloc[i,j] == "K":
        counter_real[8] += 1
      if real_seq.iloc[i,j] == "L":
        counter_real[9] += 1
      if real_seq.iloc[i,j] == "M":
        counter_real[10] += 1
      if real_seq.iloc[i,j] == "N":
        counter_real[11] += 1
      if real_seq.iloc[i,j] == "P":
        counter_real[12] += 1
      if real_seq.iloc[i,j] == "Q":
        counter_real[13] += 1
      if real_seq.iloc[i,j] == "R":
        counter_real[14] += 1
      if real_seq.iloc[i,j] == "S":
        counter_real[15] += 1
      if real_seq.iloc[i,j] == "T":
        counter_real[16] += 1
      if real_seq.iloc[i,j] == "V":
        counter_real[17] += 1
      if real_seq.iloc[i,j] == "W":
        counter_real[18] += 1
      if real_seq.iloc[i,j] == "Y":
        counter_real[19] += 1

  real.append(counter_real)



pred = []
for i in range(0,df_pred.shape[0]):
  counter_pred = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
  for j in range(0,df_pred.shape[1]):
    if isinstance(df_pred.iloc[i,j],str):
      continue
    elif df_pred.iloc[i,j] > 0:
      if real_seq.iloc[i,j] == "A":
        counter_pred[0] += 1
      if real_seq.iloc[i,j] == "C":
        counter_pred[1] += 1
      if real_seq.iloc[i,j] == "D":
        counter_pred[2] += 1
      if real_seq.iloc[i,j] == "E":
        counter_pred[3] += 1
      if real_seq.iloc[i,j] == "F":
        counter_pred[4] += 1
      if real_seq.iloc[i,j] == "G":
        counter_pred[5] += 1
      if real_seq.iloc[i,j] == "H":
        counter_pred[6] += 1
      if real_seq.iloc[i,j] == "I":
        counter_pred[7] += 1
      if real_seq.iloc[i,j] == "K":
        counter_pred[8] += 1
      if real_seq.iloc[i,j] == "L":
        counter_pred[9] += 1
      if real_seq.iloc[i,j] == "M":
        counter_pred[10] += 1
      if real_seq.iloc[i,j] == "N":
        counter_pred[11] += 1
      if real_seq.iloc[i,j] == "P":
        counter_pred[12] += 1
      if real_seq.iloc[i,j] == "Q":
        counter_pred[13] += 1
      if real_seq.iloc[i,j] == "R":
        counter_pred[14] += 1
      if real_seq.iloc[i,j] == "S":
        counter_pred[15] += 1
      if real_seq.iloc[i,j] == "T":
        counter_pred[16] += 1
      if real_seq.iloc[i,j] == "V":
        counter_pred[17] += 1
      if real_seq.iloc[i,j] == "W":
        counter_pred[18] += 1
      if real_seq.iloc[i,j] == "Y":
        counter_pred[19] += 1

  pred.append(counter_pred)


real_contacts = pd.DataFrame(real,columns = ("A","C","D","E","F","G","H","I","K","L","M","N","P","Q","R","S","T","V","W","Y"))
real_contacts.index = df_real.index

pred_contacts = pd.DataFrame(pred,columns = ("A","C","D","E","F","G","H","I","K","L","M","N","P","Q","R","S","T","V","W","Y"))
pred_contacts.index = df_real.index

In [ ]:
real_contacts

In [ ]:
pred_contacts

In [ ]:
import pandas as pd

counter_avgs = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
for i in range(0,real_contacts.shape[0]):
  for j in range(0,real_contacts.shape[1]):
      counter_avgs[j] += real_contacts.iloc[i,j]

real_avgs = pd.DataFrame(counter_avgs,columns = ["Averages"])
real_avgs.index = real_contacts.columns

real_avgs # real contacts

In [ ]:
real_contacts / contact_AAs

In [ ]:
import pandas as pd

counter_avgs2 = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
for i in range(0,pred_contacts.shape[0]):
  for j in range(0,pred_contacts.shape[1]):
      counter_avgs2[j] += pred_contacts.iloc[i,j]

pred_avgs = pd.DataFrame(counter_avgs2,columns = ["Averages"])
pred_avgs.index = real_contacts.columns

pred_avgs # predicted contacts

In [ ]:
pred_contacts/contact_AAs

In [ ]:
import pandas as pd

counter_avgs3 = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
for i in range(0,contact_AAs.shape[0]):
  for j in range(0,contact_AAs.shape[1]):
      counter_avgs3[j] += contact_AAs.iloc[i,j]

available_avgs = pd.DataFrame(counter_avgs3,columns = ["Averages"])
available_avgs.index = real_contacts.columns

available_avgs # available contacts

In [ ]:
real_avgs/available_avgs # real contacts / available


In [ ]:
pred_avgs/available_avgs # pred contacts / available


In [ ]:
difference = (real_avgs/available_avgs) - (pred_avgs/available_avgs)
difference

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

plt.figure(figsize=[20,5])
plt.bar(difference.index, difference['Averages'], color = 'pink')
plt.title('Comparing the average number of each amino acid, by contact, for real - predicted (positive = more real; negative = more pred)')
plt.xlabel('Amino Acids')
plt.ylabel('Average')

plt.yticks(np.arange(-0.5, 0.6, 0.1))  # set y-axis ticks

plt.axhline(0,color = 'black')


plt.grid(True)
plt.show

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

plt.figure(figsize=[20,5])
plt.bar(available_avgs.index, available_avgs['Averages'], color = 'pink')
plt.title('number of available contacts by amino acid')
plt.xlabel('Amino Acids')
plt.ylabel('Average')

plt.yticks(np.arange(0, 501, 50))  # set y-axis ticks



plt.grid(True)
plt.show
